<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library

# Example: Access data and meta data


## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

## Import the library

In [ ]:
from icoscp.cpb.dobj import Dobj

 ## Create a data object

In [ ]:
pid = 'https://meta.icos-cp.eu/objects/ZPSu9wc3rK6ZRip8LT8jV--f'
dobj = Dobj(pid)

## Meta data
Ask for information (meta data) about the data set. For a full list of available<br>
attributes, please check the [documentation](https://icos-carbon-portal.github.io/pylib/modules/#dobj).

In [ ]:
dobj.colNames

In [ ]:
dobj.citation

In [ ]:
dobj.licence

In [ ]:
# all meta data
dobj.meta

## Get the data

In [ ]:
dobj.data.head()

## Make a plot

In [ ]:
dobj.data.plot(x='TIMESTAMP', y='ch4', grid=True)

## Make a better plot

In [ ]:
unit = dobj.variables[dobj.variables.name == 'ch4'].unit.values[0]
name = dobj.station['org']['name']
uri = dobj.station['org']['self']['uri']
title = f"{name} \n {uri}"

In [ ]:
import matplotlib.pyplot as plt
plot = dobj.data.plot(x='TIMESTAMP', y='ch4', grid=True, title=title)
plot.set(ylabel=unit)
plt.show()